# Technischer Vorbeschrieb
Ein technischer Vorbeschrieb ist eine strukturierte Dokumentation, die die wesentlichen technischen Merkmale, Funktionen und Spezifikationen eines Produkts beschreibt. Er dient als Grundlage für die Planung, Entwicklung und Kommunikation zwischen Herstellern, Lieferanten und Kunden. Der Vorbeschrieb fasst technische Details prägnant zusammen und hilft dabei, Anforderungen und Erwartungen klar zu definieren.

Diese Informationen stammen aus Herstellerdokumenten in verschiedenen Formaten (z. B. PDF-Dateien, Websites, Planungsportale) und werden auf Basis der Kriterien für einen technischen Vorbeschrieb zusammengefasst.

Im Rahmen des Digitalisierungsprojekts wurde zunächst mit der Verarbeitung von PDF-Dateien begonnen, wobei die Pipeline zukünftig auf weitere Datenquellen erweitert werden kann.

In [ ]:
# load data
import fitz
import json
import os
import re
from PIL import Image
import pytesseract
import io
from typing import Tuple, List, Dict, Union
import base64
from io import BytesIO

# Ollama
import ollama
ollama.pull(model='llama3.2')

# template
from docx import Document
from docx.shared import Pt, RGBColor, Inches
from docx.oxml.ns import nsdecls, qn
from docx.oxml import parse_xml, OxmlElement
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

# https://github.com/UB-Mannheim/tesseract/wiki
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\sandra.nuissl\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'


## 1. Load Data + Extraktion

Zu Beginn werden die Inhalte der PDF extrahiert und in einzelnen Listen gespeichert. Dies ist ein wesentlicher Schritt für die anschließende Verarbeitung. Für die Extraktion stehen verschiedene Bibliotheken zur Verfügung, die jeweils ihre eigenen Vor- und Nachteile haben. Eine Übersicht dieser Punkte ist in der untenstehenden Tabelle aufgeführt. Da Fitz zudem in der Lage ist, mit Bilddaten zu arbeiten und sich mit OCR-Technologie kombinieren lässt, wurde diese Bibliothek für den Prozess gewählt.

| **Kriterium**                        | **PyPDFLoader (PyPDF2)**                                      | **Fitz (PyMuPDF)**                                      |
|--------------------------------------|---------------------------------------------------------------|---------------------------------------------------------|
| **Text-Extraktion**                  | Gut bei strukturiertem, einfachem Text                         | Besser bei komplexen Layouts, aber teilweise fragmentiert|
| **Layout-Erhaltung**                 | Schwach bei komplexen Layouts (Tabellen, Spalten, Grafiken)    | Gut bei der Erhaltung von Layout und Struktur           |
| **Bilder und Medien**                | Extrahiert nur Text, keine Bilder oder eingebetteten Inhalte   | Kann Bilder, Text, Annotationen und andere Inhalte extrahieren |
| **OCR-Unterstützung**                | Keine native Unterstützung                                     | Unterstützt Bildextraktion, ideal in Kombination mit OCR |
| **Performance**                      | Gute Performance bei einfachen PDFs                           | Sehr schnell, auch bei großen und komplexen PDFs        |
| **Textcodierung**                    | Probleme bei PDFs mit ungewöhnlicher Textcodierung (Fonts)     | Besser bei schwierigen Textcodierungen                   |
| **Komplexität**                      | Einfach zu verwenden                                          | Mächtig, aber etwas komplexere API                      |
| **Ressourcenverbrauch**              | Leichtgewichtig                                                | Höherer Speicherverbrauch durch zusätzliche Funktionen   |
| **Anwendungsfälle**                  | Eher für einfache, textbasierte PDFs                           | Geeignet für komplexe PDFs mit Bildern, Tabellen und Spalten |
| **Metadaten**                        | Kann Metadaten (Titel, Autor, Annotationen) extrahieren         | Kann ebenfalls Metadaten extrahieren, aber auch Annotationen |


In [43]:
# Extract the text of the PDF
def extract_text_from_pdf(pdf_document: fitz.Document) -> list[str]:
    """Extracts the text from a PDF document and returns it as a list of texts and as context.

    Args:
        pdf_document: The PDF document from which the text is to be extracted. 

    Returns:
        context: A list containing a single string where all the text from the PDF pages is joined 
                together into a single context string.                         
    """

    # empty list
    texts = []

    # iterierate through the pages
    for page_num in range(len(pdf_document)):
        # load the current page
        page = pdf_document.load_page(page_num)

        # extract the text and append to the list (+ replace paragraph signs)
        text = page.get_text("text")
        text = text.replace("\n", "")
        texts.append(text)

    # join the texts to one context list
    context = [" ".join(texts)]

    return context

In [44]:
# Exctract the images from the PDF
def extract_images_from_pdf(pdf_document: fitz.Document) -> Tuple[str, Union[Image.Image, str]]:
    """ Extracts images from a PDF document and performs OCR to extract text from those images.

    Args:
        pdf_document: The PDF document from which the text is to be extracted. 

    Returns:
        images: A list of images extracted from the PDF.
        ocr_texts: A list of text strings extracted from the images.
    """

    # empty lists
    images = []
    ocr_texts = []

    # iterierate through the pages
    for page_num in range(len(pdf_document)):
        # load the current page
        page = pdf_document.load_page(page_num)

        # extract images from PDF
        image_list_per_page = page.get_images(full=True)
        for img_index, img in enumerate(image_list_per_page):
            xref = img[0]                                       # reference number
            base_image = pdf_document.extract_image(xref)       # extract image by reference number 
            image_bytes = base_image["image"]                   # extract image content

            # save image at the list as PIL
            image = Image.open(io.BytesIO(image_bytes))
            images.append(image)
            
            # extract content in image with OCR
            if image_bytes:
                try:
                    ocr_text = pytesseract.image_to_string(image)
                    ocr_texts.append(ocr_text)
                except Exception as e:
                    print(f"Error opening the image on page {page_num + 1}, Image {img_index + 1}: {e}")
            else:
                pass

    return images, ocr_texts


In [45]:
# Extract data from the PDF
def process_pdf(pdf_path: str) -> Dict[str, Union[Image.Image, str]]:
    """Processes a PDF document to extract text, context, images, and OCR text from images.

    Args:
        pdf_path: The file path to the PDF document to be processed.

    Returns: 
        document_data: A dictionary containing the 'context', 'images' and 'ocr_texts'
    """

    # open PDF file
    pdf_document = fitz.open(pdf_path)

    # extract text
    context = extract_text_from_pdf(pdf_document)

    # extract images
    images, ocr_texts = extract_images_from_pdf(pdf_document)

    # close PDF file
    pdf_document.close()

    # store results in a dictionary
    document_data = {
        "context": context,
        "images": images,
        "ocr_texts": ocr_texts
    }

    return document_data


In [46]:
# set path
pdf_path = "Data\Produktdatenblatt_Migration_SE.pdf"
#pdf_path = "Data\Factsheet AM Chair-DE (1).pdf"
#pdf_path = "Data\Technisches Datenblatt SET 120x200 cm Think Tank.pdf"

In [47]:
# example
document_data = process_pdf(pdf_path)

print(document_data.keys())
document_data["context"]

dict_keys(['context', 'images', 'ocr_texts'])


['Migration SEHöhenverstellbare Schreibtische,  Benches und Besprechungstische Die Migration SE Produktfamilie bietet eine Auswahl hochwertiger, flexibler höhenverstellbarer Produkte mit gutem Preis-Leistungs-Verhältnis. Sie können in vielen unterschiedlichen Applikationen eingesetzt werden und tragen zur Steigerung des Wohlbefindens bei. Die schlichte, bewährte Produktreihe gibt den Angestellten die Möglichkeit, selbst zu entscheiden, wann sie im Sitzen bzw. Stehen arbeiten möchten. Ergonomie und LeistungsstärkeWenn die Mitarbeitenden ins Büro kommen, wollen sie nicht nur gemeinsam arbeiten und sich mit Kolleg*innen austauschen, sondern sie erwarten auch leistungsstarke Produkte, die sich auch dadurch auszeichnen, dass man die Höhe selbst einstellen kann. Beim Design der Migration SE-Produktfamilie lag der Fokus auf Flexibilität, Preis/Leistung und dem Nutzer-Wohlbefinden.ProduktlinieSchreibtische + BenchesDie höhenverstellbaren Schreibtische und Benches von Migration SE bieten langfr

## 2. Zusammenfassung
Für Doku: https://ollama.com/library/llama3.1

Mithilfe des Sprachmodells Ollama wird die geladenen PDF Datei in Bezug auf die Kriterien eines technischen Vorbeschriebs zusammengefasst. Dieser Vorgang gliedert sich in folgende Unterschritte auf:

- Ermittlung der Kriterien auf Basis des Möbelstücks
- Extraktion der Inhalte zu der Liste der Kriterien

### Ermittlung der Kriterien auf Basis des Möbelstücks
Mithilfe des Sprachmodells wird im ersten Schritt eine Liste der Kriterien erstellt, die für ein ausgewähltes Möbelstück in einem technischen Vorbeschrieb erwähnt werden sollten. Dieser Ansatz ist wichtig, da möglicherweise nicht alle relevanten Kriterien in einem einzelnen Dokument behandelt werden. Auf diese Weise können fehlende Erläuterungen identifiziert und dere Inhalte ergänzt werden, um eine vollständige und präzise Beschreibung sicherzustellen.

In [48]:
text_to_summarize = document_data["context"]

In [99]:
modelfile_furniture = f'''
Nenne den Möbeltyp, welcher im Text beschrieben wird in max. 2 Wörtern {text_to_summarize}?
Verwende keine Produktnamen oder Firmennamen.
Antworte mit nur 1 Subjektiv und nur einem erläuternden Adjektiv ohne Artikel
Beispiele richtig: "Ergonomischer Schreibtischstuhl", "Höhenverstellbarer Schreibtisch", "Flexibler Besprechungstisch", ...
Beispiele falsch: "Migration SE", "Steelcase", ...
Antwortlänge = 2 Wörter
'''

furniture = ollama.generate(model='llama3.2', prompt=modelfile_furniture).response

# Ausgabe
furniture

'Ergonomischer Schreibtisch'

In [134]:
# Create new modified model
modelfile_criteria = f'''
SYSTEM Deine Aufgabe ist es, Schlagwörter für die Kriterien, welche in einem technischen Vorbeschrieb für die Ausschreibung von einem vorgegebenen Möbelstück berücksichtigt werden müssen, aufzulisten\
Nenne alle Kriterien, welche für das Möbelstück von bedeutung sind\
Die Liste soll NUR die Schlagwörter enthalten, die als strukturierende Kategorien dienen könnten, ohne weitere Details oder erklärende Texte.\
Hinweise:\
- Keine numerischen Aufzählungen\
- Nur Oberpunkte und Keine Unterpunkte\
- Keine Anführungszeichen, wie "" oder ''\
- keine Doppelpunkte, Klammern oder Spiegelstriche\
- Keine Absätze\
- keine erläuternden Sätze\
- Keine einleitenden Worte wie "Hier ist die Liste mit den Schlagwörtern:"\
- Keine abschließenden Worte wie "hier sind die Informationen ..." oder "ich kann dir noch weitere Informationen liefern ..."\
- Mögliche Begriffe können sein: "Materialien", "Mechanik", "Nachhaltigkeit", "Zertifizierungen" usw.\
- Der erste Punkt soll immer "Allgemein" sein, da dieser dann im nachgang befüllt wird\
Erstelle eine Aufzählung aller Überschriften der technischen Kriterien für den technischen Vorbeschrieb welche für die Beschreibunge dieses Möbelstücks nötig sind: {furniture}
'''

criteria = ollama.generate(model='llama3.2', prompt=modelfile_criteria).response

# Ausgabe
criteria.split("\n")

['Allgemein',
 'Materialien',
 'Mechanik',
 'Nachhaltigkeit',
 'Zertifizierungen',
 'Herstellungsprozess',
 'Produktionsstandort',
 'Größe und Abmessung',
 'Durchmesser (wenn relevant)',
 'Gewicht',
 'Farbe und Oberflächenveredelung',
 'Sicherheitseigenschaften',
 'Elektrische Anforderungen',
 'Wasserdichte Ausführung',
 'Feuerfestigkeit',
 'UV-Beschutz',
 'Chemikalienschutz',
 'Staub- und Schmutzabsonderung',
 'Abfallbeseitigungsfähigkeit']

In [ ]:
# Entferne Sonderzeichen, trenne nach Zeilenumbrüchen und speichern in eine Liste
criteria_list = [re.sub(r"[^a-zA-ZäöüÄÖÜß\s-]", "", line).strip() for line in criteria.split("\n")]

# Ausgabe
print(criteria_list)

['Allgemein', 'Materialien', 'Mechanik', 'Nachhaltigkeit', 'Zertifizierungen', 'Herstellungsprozess', 'Produktionsstandort', 'Größe und Abmessung', 'Durchmesser wenn relevant', 'Gewicht', 'Farbe und Oberflächenveredelung', 'Sicherheitseigenschaften', 'Elektrische Anforderungen', 'Wasserdichte Ausführung', 'Feuerfestigkeit', 'UV-Beschutz', 'Chemikalienschutz', 'Staub- und Schmutzabsonderung', 'Abfallbeseitigungsfähigkeit']


### Extraktion der Inhalte zu der Liste der Kriterien

Hier wurde mit der Verwendung der richtige Sprache experimentiert. Erstaunlicherweise waren für diesen Implementierungsschritt die Ergebnisse mit einem englischsprachigen Prompt besser als mit einem deutschen Prompt. Aus diesem Grund wurde für die Implementierung der englischsprachige Prompt verwendet.

In [ ]:
# Pfad zur JSON-Datei
example_json_path = './Data/example.json'

# Beispielzusammenfassungen in einem strukturierten Format (Json file "examples" laden)
with open(example_json_path, 'r', encoding='utf-8') as file:
    examples = json.load(file)

In [138]:
messages = f'''
SYSTEM deine Aufgabe ist, es Texte für den technischen Vorbeschrieb von Ausschreibungen für Möbelstücken anhand des gegebenen PDF Dokuments zu erstellen.\
Hierbei handelt es sich nicht um eine Bewerbung des Produktes, sondern um eine neutrale und fachliche Aufführung der technischen Voraussetzungen.\
Ähnlich wie in diesen Beispieltexten:\
{examples}\
Wichtig: Nenne KEINE Firmennamen, KEINE Produktnamen\
Beschränke dich ausschließlich auf die Aufzählung der Inhalte aus dem Dokument\
Hinweise:\
- Keine numerischen Aufzählungen\
- Keine Escape-Sequenz für Tabulator wie t+ \
- Keine Anführungszeichen\
- keine Doppelpunkte, Klammern oder Spiegelstriche\
- Keine Absätze\
- keine Formatierungen (wie dick gedruckte Schrift durch ** gekennzeichnet)\
- Keine einleitenden Worte wie "Hier sind die extrahierten Inhalte der Textabschnitte zum Thema ..."\
- Keine abschließenden Worte wie "zu beachten ist ..." oder "Diese Informationen enthalten technische Spezifikationen ..."
Der bereitgestellte Text ist {text_to_summarize}.\
Extrahiere aus dem bereitgestellten Text alle Inhalte der Textabschnitte, die sich auf das Thema {criteria_list} bezieht.\
Lasse irrelevante Abschnitte aus. Konzentriere dich auf Informationen, die direkt oder indirekt mit {criteria_list} in Zusammenhang stehen,\
und stelle diese technisch präzise dar.\
Wichtig: Nenne KEINE Firmennamen, KEINE Produktnamen'''

summary = ollama.generate(
    model='llama3.2',
    prompt=messages
)

bulletpoints = summary['response'].split("\n")
bulletpoints

['Hier sind die ausgewählten Inhalte:',
 '',
 '**Allgemein**',
 '',
 '* Migration SE wird von Steelcase in Rosenheim für den EMEA-Markt hergestellt.',
 '* Die Produktionsstätte ist ISO 14001, ISO 45001, EMAS und PEFC zertifiziert.',
 '',
 '**Materialien**',
 '',
 '* Oberflächenmaterialien: Blauer Engel RAL-UZ 38SCS Indoor Advantage Gold PEFC',
 '* Deklarationen: SCS Indoor Advantage Gold, EPD (Umweltprodukterklärung)',
 '',
 '**Mechanik**',
 '',
 '* Höhenverstellbare Ausführung',
 '',
 '**Nachhaltigkeit**',
 '',
 '* ISO 14001, ISO 45001, EMAS, PEFC Zertifizierungen',
 '* ESD-freundliche Oberflächenmaterialien',
 '',
 '**Zertifizierungen**',
 '',
 '* SCS Indoor Advantage Gold',
 '* EPD (Umweltprodukterklärung)',
 '* ISO 14001',
 '* ISO 45001',
 '* EMAS',
 '* PEFC',
 '',
 '**Herstellungsprozess**',
 '',
 '* Herstellung in Rosenheim für den EMEA-Markt',
 '',
 '**Größe und Abmessung**',
 '',
 '* T-Fußgestell: 2000, 2200, 2400 mm',
 '* 4-Fußgestell: 2800, 3200 mm',
 '* Tiefe: 1000 mm (T-Fuß

In [144]:
messages = f'''
SYSTEM Your task is to create texts for the technical pre-description of tender documents for furniture based on the provided PDF document.\
This is not a product advertisement but a neutral and technical presentation of the requirements.\
Similar to these sample texts:\
{examples}\
Important: Do not mention company names or product names.\
Focus exclusively on listing the contents of the document.\
Write the text in the "should" perseptive and not in the "is" perspective\
Notes:\
- No numerical bullet points\
- No escape sequences for tabs like t+\
- No quotation marks\
- No colons, parentheses, or dashes\
- No paragraphs\
- No formatting (e.g., bold text marked with **)\
- No introductory phrases like "Here are the extracted contents of the text sections on the topic ..."\
- No concluding phrases like "It should be noted ..." or "This information contains technical specifications ..."
The provided text is {text_to_summarize}.\
Extract from the provided text all content from the sections that relate to the topic {criteria_list}.\
Exclude irrelevant sections. Focus on information that is directly or indirectly related to {criteria_list},\
and present it in a technically precise manner.\
Important: Do not mention company names or product names.
'''

summary = ollama.generate(
    model='llama3.2',
    prompt=messages
)

bulletpoints = summary['response'].split("\n")
bulletpoints

['**Allgemein**',
 '',
 '* Das Produkt ist ein Modul für moderne Arbeitsformen',
 '* Es wurde entwickelt, um eine optimale Arbeitssituation zu bieten',
 '* Es bietet eine Vielzahl von Funktionen und Möglichkeiten für die Anpassung an individuelle Bedürfnisse',
 '',
 '**Materialien**',
 '',
 '* Oberflächenmaterialien: Melaminplatten, Hartholz-Klone, Linoleumplatten',
 '* Farben: Standardfarben, Akzentfarben, Spezialfarben',
 '',
 '**Mechanik**',
 '',
 '* T-Fußgestell und 4-Fußgestell verfügbar',
 '* Höhenverstellbare Version erhältlich',
 '* Durchmesser der Basis: je nach Modell variieren',
 '',
 '**Nachhaltigkeit**',
 '',
 '* Produkt wird von einem zertifizierten Hersteller hergestellt',
 '* Verwendung von recycelten Materialien und minimalen Abfall',
 '* Umweltprodukterklärung (EPD) erhältlich',
 '',
 '**Zertifizierungen**',
 '',
 '* ISO 14001, ISO 45001, EMAS, PEFC zertifiziert',
 '* GS Zertifikat für Indoor Air Quality',
 '* andere Zertifikate verfügbar',
 '',
 '**Herstellungsprozes

Strukturierte Zusammenfassung erstellen und speichern
Speichere alle Zusammenfassungen in einer Liste oder einem Dictionary. Dabei kann jedes Kriterium als Schlüssel dienen, und die Zusammenfassung als Wert:

In [ ]:
bulletpoints_dict = {}
current_key = None

for line in bulletpoints:
    line = line.strip()
    # Skip empty line
    if not line:
        continue
    
    # Check if there is a key
    if line.startswith("**") and line.endswith("**"):
        current_key = re.sub(r"[*]", "", line).strip()
        bulletpoints_dict[current_key] = []
    elif current_key:
        line = re.sub(r"[*]", "", line).strip()
        bulletpoints_dict[current_key].append(line)

# Ausgabe
bulletpoints_dict

{'Allgemein': ['Das Produkt ist ein Modul für moderne Arbeitsformen',
  'Es wurde entwickelt, um eine optimale Arbeitssituation zu bieten',
  'Es bietet eine Vielzahl von Funktionen und Möglichkeiten für die Anpassung an individuelle Bedürfnisse'],
 'Materialien': ['Oberflächenmaterialien: Melaminplatten, Hartholz-Klone, Linoleumplatten',
  'Farben: Standardfarben, Akzentfarben, Spezialfarben'],
 'Mechanik': ['T-Fußgestell und 4-Fußgestell verfügbar',
  'Höhenverstellbare Version erhältlich',
  'Durchmesser der Basis: je nach Modell variieren'],
 'Nachhaltigkeit': ['Produkt wird von einem zertifizierten Hersteller hergestellt',
  'Verwendung von recycelten Materialien und minimalen Abfall',
  'Umweltprodukterklärung (EPD) erhältlich'],
 'Zertifizierungen': ['ISO 14001, ISO 45001, EMAS, PEFC zertifiziert',
  'GS Zertifikat für Indoor Air Quality',
  'andere Zertifikate verfügbar'],
 'Herstellungsprozess': ['Produkt wird in einem zertifizierten Herstellungsstandort hergestellt',
  'Verwe

In [ ]:
def get_summary(document_data: Dict[str, Union[Image.Image, str]]) -> Dict[str, List[str]]:
    """Create the summary of the uploaded pdf file.

    Args:
        document_data: A dictionary containing the 'context', 'images' and 'ocr_texts'

    Returns:
        Dict[str, List[str]]: A dictionary containing the extracted features of the categories depends on the extracted furniture
    """
        
    # Load example json
    example_json_path = './Data/example.json'
    with open(example_json_path, 'r', encoding='utf-8') as file:
        examples = json.load(file)

    # Get context
    text_to_summarize = document_data["context"]
    
    # Get furniture
    modelfile_furniture = f'''
        Nenne den Möbeltyp, welcher im Text beschrieben wird in max. 2 Wörtern {text_to_summarize}?
        Verwende keine Produktnamen oder Firmennamen.
        Antworte mit nur 1 Subjektiv und nur einem erläuternden Adjektiv ohne Artikel
        Beispiele richtig: "Ergonomischer Schreibtischstuhl", "Höhenverstellbarer Schreibtisch", "Flexibler Besprechungstisch", ...
        Beispiele falsch: "Migration SE", "Steelcase", ...
        Antwortlänge = 2 Wörter
        '''

    furniture = ollama.generate(model='llama3.2', prompt=modelfile_furniture).response

    # Get criteria
    modelfile_criteria = f'''
        SYSTEM Deine Aufgabe ist es, Schlagwörter für die Kriterien, welche in einem technischen Vorbeschrieb für die Ausschreibung von einem vorgegebenen Möbelstück berücksichtigt werden müssen, aufzulisten\
        Nenne alle Kriterien, welche für das Möbelstück von bedeutung sind\
        Die Liste soll NUR die Schlagwörter enthalten, die als strukturierende Kategorien dienen könnten, ohne weitere Details oder erklärende Texte.\
        Hinweise:\
        - Keine numerischen Aufzählungen\
        - Nur Oberpunkte und Keine Unterpunkte\
        - Keine Anführungszeichen, wie "" oder ''\
        - keine Doppelpunkte, Klammern oder Spiegelstriche\
        - Keine Absätze\
        - keine erläuternden Sätze\
        - Keine einleitenden Worte wie "Hier ist die Liste mit den Schlagwörtern:"\
        - Keine abschließenden Worte wie "hier sind die Informationen ..." oder "ich kann dir noch weitere Informationen liefern ..."\
        - Mögliche Begriffe können sein: "Materialien", "Mechanik", "Nachhaltigkeit", "Zertifizierungen" usw.\
        - Der erste Punkt soll immer "Allgemein" sein, da dieser dann im nachgang befüllt wird\
        Erstelle eine Aufzählung aller Überschriften der technischen Kriterien für den technischen Vorbeschrieb welche für die Beschreibunge dieses Möbelstücks nötig sind: {furniture}
        '''

    criteria = ollama.generate(model='llama3.2', prompt=modelfile_criteria).response
    criteria_list = [re.sub(r"[^a-zA-ZäöüÄÖÜß\s-]", "", line).strip() for line in criteria.split("\n")]

    # Get summary
    messages = f'''
        SYSTEM Your task is to create texts for the technical pre-description of tender documents for furniture based on the provided PDF document.\
        This is not a product advertisement but a neutral and technical presentation of the requirements.\
        Similar to these sample texts:\
        {examples}\
        Important: Do not mention company names or product names.\
        Focus exclusively on listing the contents of the document.\
        Write the text in the "should" perseptive and not in the "is" perspective\
        Notes:\
        - No numerical bullet points\
        - No escape sequences for tabs like t+\
        - No quotation marks\
        - No colons, parentheses, or dashes\
        - No paragraphs\
        - No formatting (e.g., bold text marked with **)\
        - No introductory phrases like "Here are the extracted contents of the text sections on the topic ..."\
        - No concluding phrases like "It should be noted ..." or "This information contains technical specifications ..."
        The provided text is {text_to_summarize}.\
        Extract from the provided text all content from the sections that relate to the topic {criteria_list}.\
        Exclude irrelevant sections. Focus on information that is directly or indirectly related to {criteria_list},\
        and present it in a technically precise manner.\
        Important: Do not mention company names or product names.
        '''

    summary = ollama.generate(model='llama3.2', prompt=messages)
    bulletpoints = summary['response'].split("\n")
    
    # Save in dictionary
    bulletpoints_dict = {}
    current_key = None

    for line in bulletpoints:
        line = line.strip()
        # Skip empty line
        if not line:
            continue
        
        # Check if there is a key
        if line.startswith("**") and line.endswith("**"):
            current_key = re.sub(r"[*]", "", line).strip()
            bulletpoints_dict[current_key] = []
        elif current_key:
            line = re.sub(r"[*]", "", line).strip()
            bulletpoints_dict[current_key].append(line)

    return bulletpoints_dict


## 4. Erstellen des Templates

Bild wird aus dem System heruntergeladen und eingefügt

In [158]:
# input text for the formular
massnahmennummer = "test"
vergabenummer = "2024000185"
massnahme = "Möbelierung des Gebäudes Freyeslebenstr. 1 für die Zentrale Universitätsverwaltung der FAU"
leistung = "Lose Möbelierung ZUV LOS 2"

produkt = furniture
positionsnummer = "2.3"

introduction = 'Das anzubietende Highbacksofa-System soll durch seine große Produktfamilie viele Nutzungsmöglichkeiten bieten, die von Rückzug, über offene und dynamische Zonierungen für Mittelzonen, abbildbaren Raumstrukturen, bis hin zu eleganten und bequemen Kommunikationsbereichen reichen. '
hint = 'Die angegebenen „ca.“ Maße und Werte sind Richtwerte, geringfügige Abweichungen bis maximal +/- 5%, die die Beschaffenheit und Funktion nicht beeinträchtigen, sind zulässig. Größere Abweichungen führen zum Ausschluss des Angebotes. Nicht abweichen dürfen die Außenmaße, die ohne „ca. Angaben“ vorgegeben werden.'

image = 'Data\picture.png'

In [165]:
def create_document(massnahmennummer: str,
                    vergabenummer: str,
                    massnahme: str,
                    leistung: str,
                    produkt: str,
                    positionsnummer: str,
                    introduction: str,
                    hint: str,
                    image: str,
                    output_path: str,
                    bulletpoints_dict: Dict[str, List[str]]) -> None:

    # Create a new document
    doc = Document()

    # Adjust page margins
    sections = doc.sections
    for section in sections:
        section.top_margin = Pt(60)
        section.bottom_margin = Pt(40)
        section.left_margin = Pt(60)
        section.right_margin = Pt(80)

    # Style font
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Arial'
    font.size = Pt(10)

    # Create the header
    header = doc.sections[0].header
    paragraph_header = header.paragraphs[0]
    paragraph_header.text = (f"Maßnahmenummer: {massnahmennummer}\t \t Vergabenummer: {vergabenummer} \n"
                             f"Maßnahme: {massnahme} \nLeistung: {leistung}")
    paragraph_header.style = doc.styles["Header"]
    paragraph_header.style.font.size = Pt(9)

    # Add headline
    paragraph_headline = doc.add_heading(level=1)
    run_headline = paragraph_headline.add_run(f'Technischer Vorbeschrieb {produkt} (Pos. {positionsnummer})\n')
    run_headline.font.name = 'Arial'
    run_headline.font.size = Pt(12)
    run_headline.font.bold = True
    run_headline.font.color.rgb = RGBColor(0, 0, 0)
    run_headline.font.underline = True

    # First paragraph: introduction
    doc.add_paragraph(introduction)

    # Second paragraph: hint
    doc.add_paragraph(f'Hinweis zu Maßangaben: \n{hint} \n')

    # Third paragraph: reference
    paragraph_third = doc.add_paragraph()
    paragraph_third.add_run(f'Anforderungen an: {produkt}').bold = True
    doc.add_paragraph(
        'Das Produkt soll mindestens die folgenden Kriterien erfüllen: \n\n'
        'Alle Systemelemente sollen einer Designlinie entstammen. Das System muss serienmäßig lieferbar sein.')

    # Fourth paragraph: requirements
    paragraph_fourth = doc.add_paragraph()
    paragraph_fourth.add_run('\nAnforderung/ Kriterium').bold = True

    # List of requirements
    for category, bulletpoint in bulletpoints_dict.items():
        doc.add_paragraph(category, style='Normal')

        for point in bulletpoint:
            doc.add_paragraph(point, style='List Bullet')
        doc.add_paragraph("")

    # Image
    doc.add_paragraph('Beispielhafte Darstellung:')
    doc.add_picture(image, width=Inches(4))

    # Add footer with page numbers
    for section in doc.sections:
        footer = section.footer
        paragraph = footer.paragraphs[0]
        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        # Text
        run = paragraph.add_run("Seite ")
        run.font.size = Pt(10)

        # Current page
        fldSimple_current = OxmlElement('w:fldSimple')
        fldSimple_current.set(qn('w:instr'), 'PAGE')
        run._r.append(fldSimple_current)

        # Text
        run = paragraph.add_run(" von ")
        run.font.size = Pt(10)

        # Number for all pages
        fldSimple_total = OxmlElement('w:fldSimple')
        fldSimple_total.set(qn('w:instr'), 'NUMPAGES')
        run._r.append(fldSimple_total)

    # Save the document
    doc.save(output_path)


In [166]:
# Example usage
create_document(
    massnahmennummer=massnahmennummer,
    vergabenummer=vergabenummer,
    massnahme=massnahme,
    leistung=leistung,
    produkt=produkt,
    positionsnummer=positionsnummer,
    introduction=introduction,
    hint=hint,
    image=image,
    output_path="technischer_vorbeschrieb.docx",
    bulletpoints_dict=bulletpoints_dict
)